In [4]:
import pandas as pd
import os

from formatacao_sumulas import FootballDataProcessor
from algorithm import FootballMatchRating
import numpy as np
import warnings
warnings.filterwarnings("ignore")


id_players = pd.read_excel('id_players.xlsx', index_col=0)


data_folder = '../Data/'
file_names = ['Serie_A']
years = range(2013, 2014)

In [5]:
df_jogos_2014 = pd.read_json('../Data/Serie_A_2013_games.json')
df_jogos_2014_ = df_jogos_2014.T

for index, row in df_jogos_2014_.iterrows():
    processor = FootballDataProcessor(df_jogos_2014_, index-1)
    final_df = processor.process(data_folder, file_names, years)
    final_df.rename(columns={'player_id': 'player'}, inplace=True)

    final_df['player'] = final_df['player'].astype('string')
    id_players['player'] = id_players['player'].astype('string')

    final_df_ = pd.merge(final_df, id_players, on='player', how='left')

    team_1_name = df_jogos_2014_.iloc[index-1,0]
    team_2_name = df_jogos_2014_.iloc[index-1,1]


    # Criar uma instância de FootballMatchRating
    rating_calculator = FootballMatchRating(final_df_, team_1_name, team_2_name)

    # Atualizar as ratings
    updated_match_data = rating_calculator.update_ratings()
    # Garantir que 'player' em updated_match_data está como string
    updated_match_data['player'] = updated_match_data['player'].astype('string')
    # Merge para obter apenas as colunas 'rating' e 'games_played' que precisam ser atualizadas
    updated_match_data.rename(columns={'player_name_x': 'player_name'}, inplace=True)
    updated_match_data.rename(columns={'time_jogador_x': 'time_jogador'}, inplace=True)

    updated_values = id_players[['player']].merge(
        updated_match_data[['player', 'rating', 'games_played', 'player_name', 'time_jogador']],
        on='player',
        how='left'
    )

    # Atualizar apenas as linhas corespondentes em id_players
    id_players.update(updated_values)

id_players


,player,rating,age,games_played,player_name,time_jogador
0,310804,1500.000000,25,50,NaN,NaN
1,177745,1500.000000,25,50,NaN,NaN
2,323637,1500.000000,25,50,NaN,NaN
3,291429,1500.000000,25,50,NaN,NaN
4,313743,1502.295225,25,51,8Matheus Matheus do Amaral Olavo,Ponte Preta / SP
...,...,...,...,...,...,...
3082,307753,1475.745431,25,66,37Rafael Rafael Gimenes da Silva,Fluminense / RJ
3083,292443,1510.240231,25,52,12Omar Omar Constante Reis ...,Bahia / BA
3084,314464,1486.968345,25,56,15Luiz Eduardo Luiz Eduardo Felix d ...,Náutico / PE
3085,714251,1500.000000,25,50,NaN,NaN


In [6]:
x = id_players.sort_values(by='rating', ascending=False)
x.to_excel('teste.xlsx')
x

,player,rating,age,games_played,player_name,time_jogador
2179,173357,1586.763249,25,72,5Michel Michel Miguel da Silva,Vitória / BA
1346,169529,1580.377320,25,72,2Ayrton Ayrton Luiz Ganino,Vitória / BA
2182,181681,1577.297642,25,64,19Cassiano Cassiano Dias Moreira,Criciúma / SC
1660,187540,1563.775779,25,61,16Lima Lucas Pedro Alves de ...,Botafogo / RJ
512,311027,1561.775628,25,64,14Emerson Emerson Palmieri dos...,Santos / SP
...,...,...,...,...,...,...
2023,135288,1404.106746,25,72,7Luis Ferna ... Luis Fernando Lojudi...,Náutico / PE
199,122995,1396.900088,25,79,10Alex Alexsandro de Souza,Coritiba / PR
646,135462,1396.608172,25,80,1Ricardo Ricardo Ferreira Berna,Náutico / PE
898,169835,1389.165413,25,82,5Edicarlos Elicarlos Souza Santos,Náutico / PE
